In [133]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
import csv

In [2]:
# date0='0709_1157'
# date1='0709_1557'
# date2='0709_1934'
# dates=[date0,date1,date2]
# plate = 13
date0='0701_1957'
date1='0701_2357'
date2='0702_0357'
date3='0702_0757'
date4='0702_1157'
date5='0702_1557'
date6='0702_1957'
date7='0702_2357'
plate=13
dates=[date0,date1,date2,date3,date4,date5,date6,date7]

In [3]:
tabs_labeled=[]
for date in dates:
    tabs_labeled.append(pd.read_csv(get_path(date,plate,True,extension='_full_labeled.csv'),
                                    converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}))

In [4]:
tabs_labeled=[]
for date in dates:
    tabs_labeled.append(pd.read_csv(f'Data/graph_{date}_{plate}_full_labeled.csv',
                                    converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}))

In [5]:
nx_graphs=[]
poss=[]
for tab in tabs_labeled:
    nx_graph,pos=generate_nx_graph(tab,labeled=True)
    nx_graphs.append(nx_graph)
    poss.append(pos)

In [6]:
nx_graph_clean=[]
for graph in nx_graphs:
    S = [graph.subgraph(c).copy() for c in nx.connected_components(graph)]
    len_connected=[len(nx_graph.nodes) for nx_graph in S]
    nx_graph_clean.append(S[np.argmax(len_connected)])

In [7]:
skeletons=[]
for nx_graph in nx_graph_clean:
    skeletons.append(generate_skeleton(nx_graph,dim=(20800, 46000)))

In [8]:
factor = 5
final_pictures = [compress_skeleton(skeletons[i],factor) for i in range(len(skeletons))]

In [9]:
connections_growth_pattern = [whole_movement_identification(nx_graph_clean[i],nx_graph_clean[i+1],poss[i],poss[i+1]) for i in range(len(dates)-1)]

0.0
0.1443001443001443
0.2886002886002886
0.4329004329004329
0.5772005772005772
0.7215007215007215
0.8658008658008658
0.0
0.1366120218579235
0.273224043715847
0.4098360655737705
0.546448087431694
0.6830601092896175
0.819672131147541
0.9562841530054644
0.0
0.12755102040816327
0.25510204081632654
0.3826530612244898
0.5102040816326531
0.6377551020408163
0.7653061224489796
0.8928571428571429
0.0
0.12330456226880394
0.2466091245376079
0.36991368680641185
0.4932182490752158
0.6165228113440198
0.7398273736128237
0.8631319358816276
0.9864364981504316
0.0
0.11614401858304298
0.23228803716608595
0.34843205574912894
0.4645760743321719
0.5807200929152149
0.6968641114982579
0.8130081300813008
0.9291521486643438
0.0
0.11013215859030837
0.22026431718061673
0.3303964757709251
0.44052863436123346
0.5506607929515418
0.6607929515418502
0.7709251101321586
0.8810572687224669
0.9911894273127754
0.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9


In [11]:
connections = [c[0] for c in connections_growth_pattern]
growth_pattern = [c[1] for c in connections_growth_pattern]

In [24]:
def pinpoint_anastomosis(nx_graph_tm1,nx_grapht,equ):
    anastomosis=[]
    origins=[]
    tips = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)==1]
    def count_neighbors_is_root(equ,nx_graph,root):
        count=0
        for neighbor in nx_graph.neighbors(root):
            if neighbor in equ.keys() and equ[neighbor]==equ[root]:
                count+=1
        return(count)
    for node in nx_grapht.nodes:
        if nx_grapht.degree(node)>=3 and node in equ.keys() and equ[node] in tips and count_neighbors_is_root(equ,nx_grapht,node)<3:
            anastomosis.append(node)
            origins.append(equ[node])
    return(anastomosis,origins)

In [78]:
interest_node = 225
for i in range(len(growth_pattern)):
    print(len(growth_pattern[i][interest_node]))

4243


KeyError: 225

In [81]:
t=0
tp1=t+1
plot_t_tp1([interest_node],[interest_node],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5,)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [80]:
plt.close('all')

In [25]:
growing_tips=[[node for node in growth_pattern[i].keys() if len(growth_pattern[i][node])>=80] for i in range(len(growth_pattern))]

In [66]:
t=0
tp1=t+1
plot_t_tp1(growing_tips[t],growing_tips[t],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5,)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
degree3_nodes = [[node for node in nx_graph.nodes if nx_graph.degree(node)>=3] for nx_graph in nx_graph_clean]

In [15]:
plot_t_tp1(degree3_nodes[t],degree3_nodes[tp1],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5)

NameError: name 't' is not defined

In [26]:
anastomosiss=[pinpoint_anastomosis(nx_graph_clean[i],nx_graph_clean[i+1], connections[i])[0] for i in range (len(dates)-1)]
origins=[pinpoint_anastomosis(nx_graph_clean[i],nx_graph_clean[i+1], connections[i])[1] for i in range (len(dates)-1)]

In [ ]:
plot_t_tp1(origins[t],anastomosiss[t],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5,time='4h')

In [22]:
def plot_t_tp1(node_list_t,node_list_tp1,pos_t,pos_tp1,imt,imtp1,relabel_t=lambda x:x,relabel_tp1=lambda x:x, shift=(0,0),compress=1,save='',time=None):
    right = 0.90
    top = 0.90
    if len(save)>=1:
        fig=plt.figure(figsize=(14,12))
        size = 10
    else:
        fig = plt.figure()
        size = 5
    ax = fig.add_subplot(111)
    ax.imshow(imtp1, cmap='gray',interpolation='none')
    ax.imshow(imt, cmap='jet', alpha=0.5,interpolation='none')
    bbox_props1 = dict(boxstyle="circle", fc="grey")
    bbox_props2 = dict(boxstyle="circle", fc="white")
    ax.text(right, top, time,
        horizontalalignment='right',
        verticalalignment='bottom',
        transform=ax.transAxes,color='white')
    for node in node_list_t:
        t = ax.text((pos_t[node][1]-shift[1])//compress, (pos_t[node][0]-shift[0])//compress, str(relabel_t(node)), ha="center", va="center",
                    size=size,
                    bbox=bbox_props1)
    for node in node_list_tp1:
        if node in pos_tp1.keys():
            t = ax.text((pos_tp1[node][1]-shift[1])//compress, (pos_tp1[node][0]-shift[0])//compress, str(relabel_tp1(node)), ha="center", va="center",
                        size=size,
                        bbox=bbox_props2)
    if len(save)>=1:
        plt.savefig(save)
        plt.close(fig)
    else:
        plt.show()

In [ ]:
t=3
tp1=t+1
plot_t_tp1([2180],[2180],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5,)

In [ ]:
plot_t_tp1(degree3_nodes[t],degree3_nodes[tp1],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5)

In [19]:
def make_growth_picture_per_tip(pixels_from_tip,pos,shape=(20700,45600),factor=10,max_growth=200,min_growth=10,per_tip=True):
    final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    number_tips = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    for tip in pixels_from_tip.keys():
        growth=len(pixels_from_tip[tip])
        x=min(round(pos[tip][0]/factor),shape[0]//factor-1)
        y=min(round(pos[tip][1]/factor),shape[1]//factor-1)
        if growth<=max_growth:
#             print(number_tips)
            if growth>=min_growth:
                number_tips[x,y]+=1
                final_picture[x,y]+=growth
#         print(growth,beginx,endx)    
#     for x in range(shape[0]//factor):
#         if x%1==0:
#             print(x/2070)
#         for y in range(shape[1]//factor):
#             beginx = x*factor
#             endx=(x+1)*factor
#             beginy = y*factor
#             endy=(y+1)*factor
#             tips_in_frame = [tip for tip in pixels_from_tip.keys() if (beginx<pos[tip][0]<endx) and (beginy<pos[tip][1]<endy)]
#             #shouls be improved, len is not a good indicator of actual length...
#             growth_in_frame = [len(pixels_from_tip[tip]) for tip in tips_in_frame]
#             final_picture[x,y]=np.mean(growth_in_frame)
    if per_tip:
        return(final_picture/(number_tips+(number_tips==0).astype(np.int)),number_tips)
    else:
        return(final_picture,number_tips)

In [35]:
final_pictures_growth = [np.log(make_growth_picture_per_tip(growth_pattern[i],poss[i],factor=500,max_growth=4000,per_tip=True,min_growth=0)[0]+1) for i in range (len(growth_pattern))]


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(final_pictures_growth[0])

In [27]:
final_pictures[2].shape

(41, 91)

In [37]:
pos_problem=poss[0][444]
xbegin=pos_problem[0]-1000
ybegin=pos_problem[1]-1000
xend=pos_problem[0]+1000
yend=pos_problem[1]+1000
skeletons_small=[]
for skeleton in skeletons:
    skeletons_small.append(skeleton[xbegin:xend,ybegin:yend])
node_smalls=[]
for i,nx_graph in enumerate(nx_graph_clean):
    node_smalls.append([node for node in nx_graph.nodes if (xbegin<poss[i][node][0]<xend and ybegin<poss[i][node][1]<yend and nx_graph.degree(node)>=1)])

In [31]:
import cv2
kernel = np.ones((5,5),np.uint8)


In [40]:
skeletons_small_dilated=[cv2.dilate(skeleton.todense().astype(np.uint8),kernel,iterations = 1) for skeleton in skeletons_small]

In [38]:
t=0
tp1=t+1

In [42]:
plot_t_tp1(node_smalls[tp1],node_smalls[tp1],poss[tp1],poss[tp1],skeletons_small_dilated[tp1],skeletons_small_dilated[tp1],shift=(xbegin,ybegin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
def plot_t_tp1(node_list_t,node_list_tp1,pos_t,pos_tp1,imt,imtp1,relabel_t=lambda x:x,relabel_tp1=lambda x:x, shift=(0,0),compress=1,save=''):
    if len(save)>=1:
        fig=plt.figure((13,12))
        size = 10
    else:
        fig = plt.figure()
        size = 5
    ax = fig.add_subplot(111)
    ax.imshow(imtp1, cmap='gray',interpolation='none')
    ax.imshow(imt, cmap='jet', alpha=0.5,interpolation='none')
    bbox_props1 = dict(boxstyle="circle", fc="grey")
    bbox_props2 = dict(boxstyle="circle", fc="white")
    for node in node_list_t:
        t = ax.text((pos_t[node][1]-shift[1])//compress, (pos_t[node][0]-shift[0])//compress, str(relabel_t(node)), ha="center", va="center",
                    size=size,
                    bbox=bbox_props1)
    for node in node_list_tp1:
        if node in pos_tp1.keys():
            t = ax.text((pos_tp1[node][1]-shift[1])//compress, (pos_tp1[node][0]-shift[0])//compress, str(relabel_tp1(node)), ha="center", va="center",
                        size=size,
                        bbox=bbox_props2)
    if len(save)>=1:
        plt.savefig(save)
        plt.close(fig)
    else:
        plt.show()